# PDF

[Portable Document Format (PDF)](https://en.wikipedia.org/wiki/PDF), ISO 32000으로 표준화된 파일 형식은 Adobe가 1992년에 문서를 제시하기 위해 개발했으며, 이는 응용 소프트웨어, 하드웨어 및 운영 시스템에 독립적인 방식으로 텍스트 서식 및 이미지를 포함합니다.

이 가이드는 `PDF` 문서를 LangChain [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 형식으로 로드하는 방법을 다룹니다. 이 형식은 다운스트림에서 사용됩니다.

LangChain은 다양한 PDF 파서와 통합됩니다. 일부는 간단하고 상대적으로 저수준이며, 다른 일부는 OCR 및 이미지 처리를 지원하거나 고급 문서 레이아웃 분석을 수행합니다. 

올바른 선택은 사용자의 애플리케이션에 따라 달라집니다.

**참고**

- [LangChain 도큐먼트](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/)

## AutoRAG 팀에서의 PDF 실험

AutoRAG 에서 진행한 실험을 토대로 작성한 순위표

아래 표기된 숫자는 등수를 나타냅니다. (The lower, the better)

| | PDFMiner | PDFPlumber | PyPDFium2 | PyMuPDF | PyPDF2 |
|----------|:---------:|:----------:|:---------:|:-------:|:-----:|
| Medical  | 1         | 2          | 3         | 4       | 5     |
| Law      | 3         | 1          | 1         | 3       | 5     |
| Finance  | 1         | 2          | 2         | 4       | 5     |
| Public   | 1         | 1          | 1         | 4       | 5     |
| Sum      | 5         | 5          | 7         | 15      | 20    |

출처: [AutoRAG Medium 블로그](https://velog.io/@autorag/PDF-%ED%95%9C%EA%B8%80-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EC%B6%94%EC%B6%9C-%EC%8B%A4%ED%97%98#%EC%B4%9D%ED%8F%89)

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

**참고**: 위의 파일은 `data` 폴더 내에 다운로드 받으세요

In [41]:
# PDF 파일 로드
# FILE_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"
FILE_PATH = "./data/2103_page5.pdf"


In [42]:
def show_metadata(docs):
    if docs:
        print("[metadata]")
        print(list(docs[0].metadata.keys()))
        print("\n[examples]")
        max_key_length = max(len(k) for k in docs[0].metadata.keys())
        for k, v in docs[0].metadata.items():
            print(f"{k:<{max_key_length}} : {v}")

## PyPDF

여기에서는 `pypdf`를 사용하여 PDF를 문서 배열로 로드하며, 각 문서는 `page` 번호와 함께 페이지 내용 및 메타데이터를 포함합니다.

In [43]:
# 설치
# !pip install -qU pypdf

In [44]:
from langchain_community.document_loaders import PyPDFLoader

# 파일 경로 설정
loader = PyPDFLoader(FILE_PATH)

# PDF 로더 초기화
docs = loader.load()

# 문서의 내용 출력
# print(docs[10].page_content[:300])  # 10페이지의 300자 내용 출력
print(docs[0].page_content[:1])

L


In [32]:
docs

[Document(metadata={'producer': 'Adobe Acrobat (64-bit) 25 Paper Capture Plug-in', 'creator': 'PyPDF', 'creationdate': '2025-08-22T12:38:36+09:00', 'author': 'Heejin Park', 'moddate': '2025-08-22T12:47:11+09:00', 'title': 'C:\\Users\\park0\\Downloads\\2103.15348v2.pdf', 'source': './data/2103_page1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='LayoutParser: A Unified Toolkit for Deep \nLearning Based Documen t Image Analysis \nIZOZ \nunr \nIt Zejiang Shen1 詞),Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain \nLee4, Jacob Carlson3, and Weining Li5 \n1 Allen Institute for AI \nshannons©allena i.org \n2 Brown University \nruochen_zhan g©brown.edu \n3 Harvard University \n{melissadell, jacob_carlson}© fas.harvard.edu \n4 University of Washington \nbcgl©cs.wash ington.edu \n5 University of Waterloo \nw422 li©u멀aterloo.ca \n[AU\n·s~] \n현\n/oo\n寸\nESI•EOINA!X1B Abstract . Recent advances in document image analysis (DIA) have been \nprimarily driven by the applica

[Document(metadata={'producer': 'Adobe Acrobat (64-bit) 25 Paper Capture Plug-in', 'creator': 'PyPDF', 'creationdate': '2025-08-22T12:38:36+09:00', 'author': 'Heejin Park', 'moddate': '2025-08-22T12:47:11+09:00', 'title': 'C:\\Users\\park0\\Downloads\\2103.15348v2.pdf', 'source': './data/2103_page1.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='LayoutParser: A Unified Toolkit for Deep \nLearning Based Documen t Image Analysis \nIZOZ \nunr \nIt Zejiang Shen1 詞),Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain \nLee4, Jacob Carlson3, and Weining Li5 \n1 Allen Institute for AI \nshannons©allena i.org \n2 Brown University \nruochen_zhan g©brown.edu \n3 Harvard University \n{melissadell, jacob_carlson}© fas.harvard.edu \n4 University of Washington \nbcgl©cs.wash ington.edu \n5 University of Waterloo \nw422 li©u멀aterloo.ca \n[AU\n·s~] \n현\n/oo\n寸\nESI•EOINA!X1B Abstract . Recent advances in document image analysis (DIA) have been \nprimarily driven by the application of neural networks. Ideally, research \noutcomes could be easily deployed in production and extended for further \ninvestigation. However, various factors like loosely organized codebases \nand sophisticated model configurations complicate the easy reuse of im\xad\nportant innovations by a wide audience. Though there have been on-going \nefforts to improve reusability and simplify deep learning (DL) model \ndevelopment in disciplines like natural language processing and computer \nvision, none of them are optimized for challenges in the domain of DIA. \nThis represents a major gap in the existing toolkit, as DIA is central to \nacademic research across a wide range of disciplines in the social sciences \nand humanities. This paper introduces LayoutParser, an open-source \nlibrary for streamlining the usage of DL in DIA research and applica\xad\ntions. The core LayoutParser library comes with a set of simple and \nintuitive interfaces for applying and customizing DL models for layout de\xad\ntection, character recognition, and many other document processing tasks. \nTo promote extensibility, LayoutParser also incorporates a community \nplatform for sharing both pre-trained models and full documen t digiti\xad\nzation pipelines. We demonstrate that LayoutParser is helpful for both \nlightweight and large-scale digitization pipelines in real-word use cases. \nThe library is publicly available at https : //layout-parser . github . io. \nKeywords: Documen t Image Analysis • Deep Learning • Layout Analysis \n• Character Recognition • Open Source library • Toolkit. \n1 Introduction \nDeep Learning(DL)-based approaches are the state-of-the-art for a wide range of \ndocumen t image analysis (DIA) tasks including documen t image classification [ 11,')]

In [33]:
# 메타데이터 출력
show_metadata(docs)

# langchain의 버전에 따라서 메타데이터 출력 형식이 다를 수 있음

[metadata]
['producer', 'creator', 'creationdate', 'author', 'moddate', 'title', 'source', 'total_pages', 'page', 'page_label']

[examples]
producer     : Adobe Acrobat (64-bit) 25 Paper Capture Plug-in
creator      : PyPDF
creationdate : 2025-08-22T12:38:36+09:00
author       : Heejin Park
moddate      : 2025-08-22T12:47:11+09:00
title        : C:\Users\park0\Downloads\2103.15348v2.pdf
source       : ./data/2103_page1.pdf
total_pages  : 1
page         : 0
page_label   : 1


### PyPDF(OCR)

일부 PDF에는 스캔된 문서나 그림 내에 텍스트 이미지가 포함되어 있습니다. `rapidocr-onnxruntime` 패키지를 사용하여 이미지에서 텍스트를 추출할 수도 있습니다.

In [10]:
# 설치
# !pip install -qU rapidocr-onnxruntime

In [54]:
# PDF 파일 로드
# FILE_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"
FILE_PATH = "./data/2103_page4.pdf"

In [56]:
# PDF 로더 초기화, 이미지 추출 옵션 활성화
loader = PyPDFLoader(FILE_PATH, extract_images=True)

# PDF 페이지 로드
docs = loader.load()

# 페이지 내용 접근
# print(docs[4].page_content[:300])
print(docs[0].page_content[:1])

4


In [51]:
docs

[Document(metadata={'producer': 'Adobe Acrobat (64-bit) 25 Paper Capture Plug-in', 'creator': 'PyPDF', 'creationdate': '2025-08-22T12:39:29+09:00', 'author': 'Heejin Park', 'moddate': '2025-08-22T12:47:30+09:00', 'title': 'C:\\Users\\park0\\Downloads\\2103.15348v2.pdf', 'source': './data/2103_page5.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='LayoutParser: A Unified Toolkit for DL-Based DIA 5 \nTable 1: Current layout detection models in the LayoutParser model zoo \nDataset I Base Model11 Large Model I Notes \nPubLayNet [38] I F / M I M I Layouts of modern scientific documents \nPRimA [3] I M I - I Layouts of scanned modern magazines and scientific reports \nNewspaper [17] I F I - I Layouts of scanned US newspapers from the 20th century \nTableBank [18] I F I F I Table region on modern scientific and business documen t \nHJDataset [31] I F / M I - I Layouts of history Japanese documen ts \n1 For each dataset , we train several models of different sizes for diffe

In [47]:
show_metadata(docs)

# langchain의 버전에 따라서 메타데이터 출력 형식이 다를 수 있음

[metadata]
['producer', 'creator', 'creationdate', 'author', 'keywords', 'moddate', 'ptex.fullbanner', 'subject', 'title', 'trapped', 'source', 'total_pages', 'page', 'page_label']

[examples]
producer        : pdfTeX-1.40.21
creator         : LaTeX with hyperref
creationdate    : 2021-06-22T01:27:10+00:00
author          : 
keywords        : 
moddate         : 2021-06-22T01:27:10+00:00
ptex.fullbanner : This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2
subject         : 
title           : 
trapped         : /False
source          : https://arxiv.org/pdf/2103.15348.pdf
total_pages     : 16
page            : 0
page_label      : 1


## PyMuPDF

**PyMuPDF** 는 속도 최적화가 되어 있으며, PDF 및 해당 페이지에 대한 자세한 메타데이터를 포함하고 있습니다. 페이지 당 하나의 문서를 반환합니다:

In [ ]:
# 설치
# !pip install -qU pymupdf

In [13]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader(FILE_PATH)

# 문서 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[10].page_content[:300])

SPRi AI Brief |  
2023-12월호
8
코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
n 코히어와 12개 기관이  광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태, 
작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시
n 대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 
구성과 계보도 추적 가능
KEY Contents
£ 데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
n AI 기업 코히어


In [14]:
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'source', 'file_path', 'total_pages', 'format', 'title', 'author', 'subject', 'keywords', 'moddate', 'trapped', 'modDate', 'creationDate', 'page']

[examples]
producer     : Hancom PDF 1.3.0.542
creator      : Hwp 2018 10.0.0.13462
creationdate : 2023-12-08T13:28:38+09:00
source       : ./data/SPRI_AI_Brief_2023년12월호_F.pdf
file_path    : ./data/SPRI_AI_Brief_2023년12월호_F.pdf
total_pages  : 23
format       : PDF 1.4
title        : 
author       : dj
subject      : 
keywords     : 
moddate      : 2023-12-08T13:28:38+09:00
trapped      : 
modDate      : D:20231208132838+09'00'
creationDate : D:20231208132838+09'00'
page         : 0


## Unstructured

[Unstructured](https://unstructured-io.github.io/unstructured/)는 Markdown이나 PDF와 같은 비구조화된 또는 반구조화된 파일 형식을 다루기 위한 공통 인터페이스를 지원합니다. 

LangChain의 [UnstructuredPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.UnstructuredPDFLoader.html)는 Unstructured와 통합되어 PDF 문서를 LangChain [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) 객체로 파싱합니다.

In [57]:
# 설치
# !pip install -qU unstructured
FILE_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"

In [58]:
from langchain_community.document_loaders import UnstructuredPDFLoader

# UnstructuredPDFLoader 인스턴스 생성
loader = UnstructuredPDFLoader(FILE_PATH)

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content[:300])

ModuleNotFoundError: No module named 'pdfminer.psexceptions'

In [ ]:
show_metadata(docs)

내부적으로 비정형에서는 텍스트 청크마다 서로 다른 "**요소**"를 만듭니다. 기본적으로 이들은 결합되어 있지만 `mode="elements"`를 지정하여 쉽게 분리할 수 있습니다.

In [ ]:
# UnstructuredPDFLoader 인스턴스 생성(mode="elements")
loader = UnstructuredPDFLoader(FILE_PATH, mode="elements")

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content)

이 특정 문서에 대한 전체 요소 유형 집합을 참조하세요

In [ ]:
set(doc.metadata["category"] for doc in docs)  # 데이터 카테고리 추출

In [ ]:
show_metadata(docs)

## PyPDFium2

In [59]:
from langchain_community.document_loaders import PyPDFium2Loader

# PyPDFium2 로더 인스턴스 생성
loader = PyPDFium2Loader(FILE_PATH)

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[10].page_content[:300])

SPRi AI Brief | 
2023-12월호
8
코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
n 코히어와 12개 기관이 광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태, 작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시
n 대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 
구성과 계보도 추적 가능
KEY Contents
£ 데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
n AI 기업 코히어(Co


c:\Users\SBA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-Us6BDj1P-py3.11\Lib\site-packages\pypdfium2\_helpers\textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [60]:
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'title', 'author', 'subject', 'keywords', 'moddate', 'source', 'total_pages', 'page']

[examples]
producer     : Hancom PDF 1.3.0.542
creator      : Hwp 2018 10.0.0.13462
creationdate : 2023-12-08T13:28:38+09:00
title        : 
author       : dj
subject      : 
keywords     : 
moddate      : 2023-12-08T13:28:38+09:00
source       : ./data/SPRI_AI_Brief_2023년12월호_F.pdf
total_pages  : 23
page         : 0


## PDFMiner

In [ ]:
from langchain_community.document_loaders import PDFMinerLoader

# PDFMiner 로더 인스턴스 생성
loader = PDFMinerLoader(FILE_PATH)

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content[:300])

# 한글 문서의 경우 띄어쓰기가 2칸씩 될 수 있으므로 주의가 필요

2023년  12월호
2023년  12월호

Ⅰ.  인공지능  산업  동향  브리프

  1.  정책/법제 

      ▹  미국,  안전하고  신뢰할  수  있는  AI  개발과  사용에  관한  행정명령  발표    ························· 1

      ▹  G7,  히로시마  AI  프로세스를  통해  AI  기업  대상  국제  행동강령에  합의 ··························· 2

      ▹  영국  AI  안전성  정상회의에  참가한  28개국,  AI  위험에  공동  


In [64]:
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'author', 'moddate', 'pdfversion', 'total_pages', 'source']

[examples]
producer     : Hancom PDF 1.3.0.542
creator      : Hwp 2018 10.0.0.13462
creationdate : 2023-12-08T13:28:38+09:00
author       : dj
moddate      : 2023-12-08T13:28:38+09:00
pdfversion   : 1.4
total_pages  : 23
source       : ./data/SPRI_AI_Brief_2023년12월호_F.pdf


**PDFMiner**를 사용하여 HTML 텍스트 생성

이 방법은 출력된 HTML 콘텐츠를 `BeautifulSoup`을 통해 파싱함으로써 글꼴 크기, 페이지 번호, PDF 헤더/푸터 등에 대한 보다 구조화되고 풍부한 정보를 얻을 수 있게 하여 텍스트를 의미론적으로 섹션으로 분할하는 데 도움이 될 수 있습니다.

In [ ]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader

# PDFMinerPDFasHTMLLoader 인스턴스 생성
loader = PDFMinerPDFasHTMLLoader(FILE_PATH)

# 문서 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content[:300])

In [ ]:
show_metadata(docs)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(docs[0].page_content, "html.parser")  # HTML 파서 초기화
content = soup.find_all("div")  # 모든 div 태그 검색

In [ ]:
import re

cur_fs = None
cur_text = ""
snippets = []  # 동일한 글꼴 크기의 모든 스니펫 수집
for c in content:
    sp = c.find("span")
    if not sp:
        continue
    st = sp.get("style")
    if not st:
        continue
    fs = re.findall("font-size:(\d+)px", st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text, cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text, cur_fs))
# 중복 스니펫 제거 전략 추가 가능성 (PDF의 헤더/푸터가 여러 페이지에 걸쳐 나타나므로 중복 발견 시 중복 정보로 간주 가능)

In [17]:
from langchain_core.documents import Document

cur_idx = -1
semantic_snippets = []
# 제목 가정: 높은 글꼴 크기
for s in snippets:
    # 새 제목 판별: 현재 스니펫 글꼴 > 이전 제목 글꼴
    if (
        not semantic_snippets
        or s[1] > semantic_snippets[cur_idx].metadata["heading_font"]
    ):
        metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
        metadata.update(docs[0].metadata)
        semantic_snippets.append(Document(page_content="", metadata=metadata))
        cur_idx += 1
        continue

    # 동일 섹션 내용 판별: 현재 스니펫 글꼴 <= 이전 내용 글꼴
    if (
        not semantic_snippets[cur_idx].metadata["content_font"]
        or s[1] <= semantic_snippets[cur_idx].metadata["content_font"]
    ):
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata["content_font"] = max(
            s[1], semantic_snippets[cur_idx].metadata["content_font"]
        )
        continue

    # 새 섹션 생성 조건: 현재 스니펫 글꼴 > 이전 내용 글꼴, 이전 제목 글꼴 미만
    metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
    metadata.update(docs[0].metadata)
    semantic_snippets.append(Document(page_content="", metadata=metadata))
    cur_idx += 1

print(semantic_snippets[4])

NameError: name 'snippets' is not defined

## PyPDF 디렉토리

디렉토리에서 PDF를 로드하세요

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# 디렉토리 경로
loader = PyPDFDirectoryLoader("data/")

# 문서 로드
docs = loader.load()

# 문서의 개수 출력
print(len(docs))

In [ ]:
# 문서의 내용 출력
print(docs[50].page_content[:300])

In [ ]:
# metadata 출력
print(docs[50].metadata)

## PDFPlumber

PyMuPDF와 마찬가지로, 출력 문서는 PDF와 그 페이지에 대한 자세한 메타데이터를 포함하며, 페이지 당 하나의 문서를 반환합니다.

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader

# PDF 문서 로더 인스턴스 생성
loader = PDFPlumberLoader(FILE_PATH)

# 문서 로딩
docs = loader.load()

# 첫 번째 문서 데이터 접근
print(docs[10].page_content[:300])

In [ ]:
show_metadata(docs)